## Goal: get SNPs both CLOSE to SJs (exonic, +- 10 bp) and within SSs themselves; only for annotated SJs

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot

p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *

In [6]:
config = load_config()
od = '../../'

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

In [ ]:
gtf_file = proc_cfg(config['ref']['gtf'], od)


,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,exon_number,exon_id,transcript_support_level,havana_transcript,hgnc_id,havana_gene,ont,protein_id,ccdsid,artif_dupl
0,GL000008.2,HAVANA,intron,84145,85442,.,+,.,ENSG00000296732.1,lncRNA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GL000008.2,HAVANA,intron,85721,88635,.,+,.,ENSG00000296732.1,lncRNA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GL000008.2,HAVANA,intron,88844,100706,.,+,.,ENSG00000296732.1,lncRNA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GL000008.2,HAVANA,intron,101630,129984,.,+,.,ENSG00000296732.1,lncRNA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GL000008.2,HAVANA,intron,130626,135133,.,+,.,ENSG00000296732.1,lncRNA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# read in ref. gtf and get SJs from there
temp = pr.read_gtf(gtf_file)

# get the introns
temp = temp.features.introns()
temp = temp.df

temp.Start = temp.Start.astype(int)
temp.End = temp.End.astype(int)
assert len(temp.loc[temp.Start>temp.End])==0

# melt to 5' and 3'
temp = temp.melt(id_vars=['Chromosome', 'Strand'],
                 value_vars=['Start', 'End'])
temp['sj_loc'] = ''
temp.loc[temp.variable=='Start', 'sj_loc'] = 'start'
temp.loc[temp.variable=='End', 'sj_loc'] = 'end'

temp.rename({'value':'Start'}, axis=1, inplace=True)
# need to verify that this is working using like one motif or something make
# sure I don't have off-by-one errors
# verified
temp['Start'] = temp.Start-2
temp['End'] = temp.Start+1

# need to verify
temp.loc[temp.sj_loc=='start', 'Start'] = temp.loc[temp.sj_loc=='start', 'Start']-9
temp.loc[temp.sj_loc=='start', 'End'] = temp.loc[temp.sj_loc=='start', 'End']+2


temp.loc[temp.sj_loc=='end', 'End'] = temp.loc[temp.sj_loc=='end', 'End']+11
temp.loc[temp.sj_loc=='end', 'Start'] = temp.loc[temp.sj_loc=='end', 'Start']

temp['len'] = temp.End-temp.Start
assert len(temp.loc[temp.len!=12]) == 0

temp = pr.PyRanges(temp)

temp.to_bed(output.bed)
